<a href="https://colab.research.google.com/github/sunghwan-moon/2021_capstone_BrainAgePrediction_by_Federated_Learning/blob/main/FL_3D_MRI_ResNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install tensorflow_federated
#!pip install nilearn
#!pip install transformations
#!pip install nest_asyncio

In [ ]:
import collections
from nilearn import image
import numpy as np
import tensorflow as tf
import tensorflow_federated as tff
import threading
from transformations import rotation_matrix
from random import gauss
from scipy.ndimage.interpolation import map_coordinates
import operator
from scipy.ndimage.interpolation import shift,rotate
import pandas as pd
import os
import scipy.ndimage as nd
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score,mean_absolute_error
import numpy as np
import matplotlib.pyplot as plt
import math
import sys
import random
import glob
import nibabel as nib
from collections import defaultdict
from tensorflow.keras.utils import Sequence
import nest_asyncio
nest_asyncio.apply()
#from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Input, add, concatenate,GlobalAveragePooling3D,Conv3D,LeakyReLU,ELU, MaxPooling3D,AveragePooling3D
from tensorflow.keras.regularizers import l2 as L2
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from tensorflow.keras.optimizers import Adam, SGD,Adagrad
from keras.engine.topology import Layer
from keras.layers import BatchNormalization
from keras.layers import Input, InputLayer, InputSpec
#from tensorflow.keras import backend as K
#from tensorflow.keras.callbacks import TensorBoard, EarlyStopping
#import tensorflow.compat.v1 as tf
# tf.compat.v1.enable_v2_behavior()
# tf.compat.v1.enable_eager_execution()

from tensorflow.compat.v1 import keras
from tensorflow.compat.v1.keras.layers import Dense, Dropout, Activation, Flatten, Input, add, concatenate,GlobalAveragePooling3D,Conv3D,LeakyReLU,ELU, MaxPooling3D,AveragePooling3D

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def getIXIData():
  data = pd.read_csv('/content/drive/MyDrive/소융_캡스톤/뇌 MRI 사진_교수님/raw/IXI.csv')
  data.drop(['ETHNIC_ID','MARITAL_ID','OCCUPATION_ID','QUALIFICATION_ID','DOB','DATE_AVAILABLE','STUDY_DATE'], axis =1, inplace = True)
  data["IXI_ID"] = data["IXI_ID"].astype(str)
  data = data.drop_duplicates(['IXI_ID'])
  data['feature'] = np.NaN
  data = data.set_index('IXI_ID')

  #RAM 초과로 인해 250개 dataset만 가지고 진행 -> 실제 데이터는 mri 폴더에 존재
  path = '/content/drive/MyDrive/소융_캡스톤/뇌 MRI 사진_교수님/dataset 일부/' 
  paths = glob.glob(path +'wm*.nii')

  id_with_data = {}
  for img_path in paths:
    img_index = img_path.split('/')[7][5:8]
    if(img_index[0] == '0' and img_index[1] == '0'):
      img_index = img_index[2]
    if(img_index[0] == '0' and img_index[1] != '0'):
      img_index = img_index[1:3]
    id_with_data[img_index] = image.get_data(img_path)

  feature_matrix = []
  drop_matrix = []
  copy_data = data.copy()

  for i in range(len(data)): 
    for key, value in id_with_data.items():
      found = False
      if(data.index[i] == key):
        found = True
        feature_matrix.append(value)
        del id_with_data[key]
        break
    if(found == False):
      drop_matrix.append(data.index[i])

  for i in range(len(drop_matrix)):
    copy_data.drop(drop_matrix[i], inplace = True)

  copy_data["feature"] = feature_matrix
  copy_data.dropna(axis=0, inplace = True)
  copy_data.reset_index(drop=True, inplace = True)

  train, val = train_test_split(copy_data, test_size = 0.2, random_state =100)

  return train,val

In [ ]:
def df_to_array(dataframe):
  temp_lst = []

  for i in range(len(dataframe)):
    feature = dataframe["feature"][i]
    temp_lst.append(feature)

  result = np.array(temp_lst)

  return result

In [ ]:
train ,val = getIXIData()

In [ ]:
train["feature"].reset_index(inplace= True, drop = True)
val["feature"].reset_index(inplace= True, drop = True)
x_train = df_to_array(train).astype(np.float32)
y_train = train["AGE"].astype(np.float16)

x_test = df_to_array(val).astype(np.float32)
y_test = np.array(val["AGE"].astype(np.float16))

num_train_image = len(x_train)
num_test_image= len(x_test)
print("Train set: ", num_train_image)
print("Test set: ", num_test_image)

Train set:  193
Test set:  49


In [ ]:
#client Data Non-iid 형태로 만들기
client_train_dataset = collections.OrderedDict()
client_test_dataset = collections.OrderedDict()
client_num = 20
for i in range(0, client_num): # Client 10명으로 제한
  client_key = "client_" + str(i+1) # client_1, client_2 .....

  # non-iid를 위한 random sampling
  front_train = random.randint(1, num_train_image) 
  rear_train = random.randint(front_train, num_train_image)

  front_test = random.randint(1, num_test_image)
  rear_test = random.randint(front_test, num_test_image)

  train_data = collections.OrderedDict((('pixels', x_train[front_train:rear_train]),('label', y_train[front_train:rear_train])))
  test_data =  collections.OrderedDict((('pixels', x_test[front_test:rear_test]),('label', y_test[front_test:rear_test])))

  client_train_dataset[client_key] = train_data
  client_test_dataset[client_key] = test_data

In [ ]:
#tensorSliceClientData를 simulation의 instance로 변환
train_dataset = tff.simulation.datasets.TestClientData(client_train_dataset)

In [ ]:
NUM_EPOCHS = 1
BATCH_SIZE = 2
SHUFFLE_BUFFER = 1
PREFETCH_BUFFER = 1

def preprocess(dataset):

  def batch_format_fn(element):
    return collections.OrderedDict(
        x=tf.reshape(element['pixels'], [-1, 121, 145, 121]),
        y=tf.reshape(element['label'], [-1, 1]))

  return dataset.repeat(NUM_EPOCHS).shuffle(SHUFFLE_BUFFER).batch(
      BATCH_SIZE).prefetch(PREFETCH_BUFFER).map(batch_format_fn)

In [ ]:
example_dataset = train_dataset.create_tf_dataset_for_client(train_dataset.client_ids[0])

In [ ]:
def make_federated_data(client_data, client_ids):
  return [
      preprocess(client_data.create_tf_dataset_for_client(x))
      for x in client_ids
  ]

In [ ]:
def make_federated_data(client_data, client_ids):
  result_list = []
  for id in client_ids:
    preprocess_data = preprocess(client_data.create_tf_dataset_for_client(id))
    sample = tf.nest.map_structure(lambda x: x.numpy(),next(iter(preprocess_data)))
    print("client ",id, " size : ", len(sample["y"]))
    result_list.append(preprocess_data)
  return result_list

In [ ]:
sample_clients = train_dataset.client_ids[0:5] #client 중에 몇명을 선발하여 Train을 할 것인가?

federated_train_data = make_federated_data(train_dataset, sample_clients)
#federated_test_data = make_federated_data(test_dataset,sample_clients)

print('Number of client datasets: {l}'.format(l=len(federated_train_data)))
print('First dataset: {d}'.format(d=federated_train_data[0]))
print("input_spec : {n}".format(n = federated_train_data[0].element_spec))

client  client_1  size :  3
client  client_2  size :  3
client  client_3  size :  3
client  client_4  size :  3
client  client_5  size :  3
Number of client datasets: 5
First dataset: <MapDataset shapes: OrderedDict([(x, (None, 121, 145, 121)), (y, (None, 1))]), types: OrderedDict([(x, tf.float32), (y, tf.float16)])>
input_spec : OrderedDict([('x', TensorSpec(shape=(None, 121, 145, 121), dtype=tf.float32, name=None)), ('y', TensorSpec(shape=(None, 1), dtype=tf.float16, name=None))])


In [ ]:
def generateAgePredictionResNet(inputShape,paddingType = 'same',initType='he_uniform',regAmount=0.00005,dropRate=0.2):
    t1Input = Input(inputShape+(1,))
    
    with tf.name_scope('ResBlock0'):
        inputs = t1Input
        features = 8
        hidden = Conv3D(features, (3, 3, 3), padding=paddingType,kernel_regularizer=L2(regAmount),kernel_initializer=initType)(inputs)
        hidden = BatchNormalization(renorm=True)(hidden)
        hidden = ELU(alpha=1.0)(hidden)
        hidden = Conv3D(features, (3, 3, 3), padding=paddingType,kernel_regularizer=L2(regAmount),kernel_initializer=initType)(hidden)
        hidden = BatchNormalization(renorm=True)(hidden)
        shortcut = Conv3D(features, (1,1,1), strides=(1,1,1), padding=paddingType,kernel_initializer=initType)(inputs)
        hidden = add([shortcut,hidden])
        outputs = ELU(alpha=1.0)(hidden)
        
    pooling = MaxPooling3D(pool_size=(2,2,2),strides=(2,2,2), padding=paddingType)(outputs)
    
    with tf.name_scope('ResBlock1'):
        inputs = pooling
        features = 16
        hidden = Conv3D(features, (3, 3, 3), padding=paddingType,kernel_regularizer=L2(regAmount),kernel_initializer=initType)(inputs)
        hidden = BatchNormalization(renorm=True)(hidden)
        hidden = ELU(alpha=1.0)(hidden)
        hidden = Conv3D(features, (3, 3, 3), padding=paddingType,kernel_regularizer=L2(regAmount),kernel_initializer=initType)(hidden)
        hidden = BatchNormalization(renorm=True)(hidden)
        shortcut = Conv3D(features, (1,1,1), strides=(1,1,1), padding=paddingType,kernel_initializer=initType)(inputs)
        hidden = add([shortcut,hidden])
        outputs = ELU(alpha=1.0)(hidden)
        
    pooling = MaxPooling3D(pool_size=(2,2,2),strides=(2,2,2), padding=paddingType)(outputs)

    with tf.name_scope('ResBlock2'):
        inputs = pooling
        features = 32
        hidden = Conv3D(features, (3, 3, 3), padding=paddingType,kernel_regularizer=L2(regAmount),kernel_initializer=initType)(inputs)
        hidden = BatchNormalization(renorm=True)(hidden)
        hidden = ELU(alpha=1.0)(hidden)
        hidden = Conv3D(features, (3, 3, 3), padding=paddingType,kernel_regularizer=L2(regAmount),kernel_initializer=initType)(hidden)
        hidden = BatchNormalization(renorm=True)(hidden)
        shortcut = Conv3D(features, (1,1,1), strides=(1,1,1), padding=paddingType,kernel_initializer=initType)(inputs)
        hidden = add([shortcut,hidden])
        outputs = ELU(alpha=1.0)(hidden)
        
    pooling = MaxPooling3D(pool_size=(2,2,2),strides=(2,2,2), padding=paddingType)(outputs)
 
    
    with tf.name_scope('ResBlock3'):
        inputs = pooling
        features = 64
        hidden = Conv3D(features, (3, 3, 3), padding=paddingType,kernel_regularizer=L2(regAmount),kernel_initializer=initType)(inputs)
        hidden = BatchNormalization(renorm=True)(hidden)
        hidden = ELU(alpha=1.0)(hidden)
        hidden = Conv3D(features, (3, 3, 3), padding=paddingType,kernel_regularizer=L2(regAmount),kernel_initializer=initType)(hidden)
        hidden = BatchNormalization(renorm=True)(hidden)
        shortcut = Conv3D(features, (1,1,1), strides=(1,1,1), padding=paddingType,kernel_initializer=initType)(inputs)
        hidden = add([shortcut,hidden])
        outputs = ELU(alpha=1.0)(hidden)
        
        
    pooling = MaxPooling3D(pool_size=(2,2,2),strides=(2,2,2), padding=paddingType)(outputs)
  
    
    with tf.name_scope('ResBlock4'):
        inputs = pooling
        features = 128
        hidden = Conv3D(features, (3, 3, 3), padding=paddingType,kernel_regularizer=L2(regAmount),kernel_initializer=initType)(inputs)
        hidden = BatchNormalization(renorm=True)(hidden)
        hidden = ELU(alpha=1.0)(hidden)
        hidden = Conv3D(features, (3, 3, 3), padding=paddingType,kernel_regularizer=L2(regAmount),kernel_initializer=initType)(hidden)
        hidden = BatchNormalization(renorm=True)(hidden)
        shortcut = Conv3D(features, (1,1,1), strides=(1,1,1), padding=paddingType,kernel_initializer=initType)(inputs)
        hidden = add([shortcut,hidden])
        outputs= ELU(alpha=1.0)(hidden)
        
    pooling = MaxPooling3D(pool_size=(2,2,2),strides=(2,2,2), padding=paddingType)(outputs)

    hidden = Flatten()(pooling)
    
    hidden = Dense(128,kernel_regularizer=L2(regAmount),kernel_initializer=initType,name='FullyConnectedLayer')(hidden)
    hidden = ELU(alpha=1.0)(hidden)
    hidden = Dropout(dropRate)(hidden)
       
    prediction = Dense(1,kernel_regularizer=L2(regAmount), name='AgePrediction')(hidden)
    model = Model(inputs=[t1Input],outputs=prediction)
    return model

In [ ]:
def model_fn():
    keras_model = generateAgePredictionResNet((121,145,121),paddingType = 'same',initType='he_uniform',regAmount=0.00005,dropRate=0.2)
    return tff.learning.from_keras_model(
      keras_model,
      input_spec=federated_train_data[0].element_spec,
      loss=tf.keras.losses.MeanAbsoluteError(),
      metrics=[tf.keras.metrics.MeanAbsoluteError()])

In [ ]:
def initialize_fn():
  model = model_fn()
  return model.trainable_variables

In [ ]:
def next_fn(server_weights, federated_dataset):
  server_weights_at_client = broadcast(server_weights)
  client_weights = client_update(federated_dataset, server_weights_at_client)
  mean_client_weights = mean(client_weights)
  server_weights = server_update(mean_client_weights)
  
  return server_weights

In [ ]:
@tf.function
def client_update(model, dataset, server_weights, client_optimizer):
  print("client_update tf.fn")
  client_weights = model.trainable_variables
  tf.nest.map_structure(lambda x, y: x.assign(y),
                        client_weights, server_weights)
  
  for batch in dataset:
      with tf.GradientTape() as tape:
        outputs = model.forward_pass(batch)

      grads = tape.gradient(outputs.loss, client_weights)
      grads_and_vars = zip(grads, client_weights)

      client_optimizer.apply_gradients(grads_and_vars)

  return client_weights

In [ ]:
@tf.function
def server_update(model, mean_client_weights):
  model_weights = model.trainable_variables
  tf.nest.map_structure(lambda x, y: x.assign(y),
                        model_weights, mean_client_weights)
  return model_weights

In [ ]:
@tff.tf_computation
def server_init():
  model = model_fn()
  return model.trainable_variables

Instructions for updating:
Colocations handled automatically by placer.


In [ ]:
@tff.federated_computation
def initialize_fn():
  return tff.federated_value(server_init(),tff.SERVER)

In [ ]:
tf_dataset_type = tff.SequenceType(model_fn().input_spec)

In [ ]:
str(tf_dataset_type)

'<x=float32[?,121,145,121],y=float16[?,1]>*'

In [ ]:
model_weights_type = server_init.type_signature.result

In [ ]:
str(model_weights_type)

'<float32[3,3,3,1,8],float32[8],float32[8],float32[8],float32[3,3,3,8,8],float32[8],float32[1,1,1,1,8],float32[8],float32[8],float32[8],float32[3,3,3,8,16],float32[16],float32[16],float32[16],float32[3,3,3,16,16],float32[16],float32[1,1,1,8,16],float32[16],float32[16],float32[16],float32[3,3,3,16,32],float32[32],float32[32],float32[32],float32[3,3,3,32,32],float32[32],float32[1,1,1,16,32],float32[32],float32[32],float32[32],float32[3,3,3,32,64],float32[64],float32[64],float32[64],float32[3,3,3,64,64],float32[64],float32[1,1,1,32,64],float32[64],float32[64],float32[64],float32[3,3,3,64,128],float32[128],float32[128],float32[128],float32[3,3,3,128,128],float32[128],float32[1,1,1,64,128],float32[128],float32[128],float32[128],float32[10240,128],float32[128],float32[128,1],float32[1]>'

In [ ]:
@tff.tf_computation(tf_dataset_type, model_weights_type)
def client_update_fn(tf_dataset, server_weights):
  model = model_fn()
  client_optimizer = tf.keras.optimizers.Adam(learning_rate=0.01) #Adam, SGD
  return client_update(model, tf_dataset, server_weights, client_optimizer)

client_update tf.fn


In [ ]:
@tff.tf_computation(model_weights_type)
def server_update_fn(mean_client_weights):
  model = model_fn()
  return server_update(model, mean_client_weights)

In [ ]:
federated_server_type = tff.FederatedType(model_weights_type, tff.SERVER)
federated_dataset_type = tff.FederatedType(tf_dataset_type, tff.CLIENTS)

In [ ]:
str(federated_dataset_type)

'{<x=float32[?,121,145,121],y=float16[?,1]>*}@CLIENTS'

In [ ]:
str(federated_server_type)

'<float32[3,3,3,1,8],float32[8],float32[8],float32[8],float32[3,3,3,8,8],float32[8],float32[1,1,1,1,8],float32[8],float32[8],float32[8],float32[3,3,3,8,16],float32[16],float32[16],float32[16],float32[3,3,3,16,16],float32[16],float32[1,1,1,8,16],float32[16],float32[16],float32[16],float32[3,3,3,16,32],float32[32],float32[32],float32[32],float32[3,3,3,32,32],float32[32],float32[1,1,1,16,32],float32[32],float32[32],float32[32],float32[3,3,3,32,64],float32[64],float32[64],float32[64],float32[3,3,3,64,64],float32[64],float32[1,1,1,32,64],float32[64],float32[64],float32[64],float32[3,3,3,64,128],float32[128],float32[128],float32[128],float32[3,3,3,128,128],float32[128],float32[1,1,1,64,128],float32[128],float32[128],float32[128],float32[10240,128],float32[128],float32[128,1],float32[1]>@SERVER'

In [ ]:
@tff.federated_computation(federated_server_type, federated_dataset_type)
def next_fn(server_weights, federated_dataset):
  server_weights_at_client = tff.federated_broadcast(server_weights)
  client_weights = tff.federated_map(
        client_update_fn, (federated_dataset, server_weights_at_client))
  mean_client_weights = tff.federated_mean(client_weights)
  server_weights = tff.federated_map(server_update_fn, mean_client_weights)

  return server_weights

In [ ]:
iterative_process = tff.learning.build_federated_averaging_process(
    model_fn,
    client_optimizer_fn=lambda: tf.keras.optimizers.Adam(learning_rate=0.02),
    server_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=1.0))

In [ ]:
str(iterative_process.initialize.type_signature)

'( -> <model=<trainable=<float32[3,3,3,1,8],float32[8],float32[8],float32[8],float32[3,3,3,8,8],float32[8],float32[1,1,1,1,8],float32[8],float32[8],float32[8],float32[3,3,3,8,16],float32[16],float32[16],float32[16],float32[3,3,3,16,16],float32[16],float32[1,1,1,8,16],float32[16],float32[16],float32[16],float32[3,3,3,16,32],float32[32],float32[32],float32[32],float32[3,3,3,32,32],float32[32],float32[1,1,1,16,32],float32[32],float32[32],float32[32],float32[3,3,3,32,64],float32[64],float32[64],float32[64],float32[3,3,3,64,64],float32[64],float32[1,1,1,32,64],float32[64],float32[64],float32[64],float32[3,3,3,64,128],float32[128],float32[128],float32[128],float32[3,3,3,128,128],float32[128],float32[1,1,1,64,128],float32[128],float32[128],float32[128],float32[10240,128],float32[128],float32[128,1],float32[1]>,non_trainable=<float32[8],float32[8],float32[8],float32[8],float32[8],float32[8],float32[8],float32[8],float32[8],float32[8],float32[16],float32[16],float32[16],float32[16],float32[16],

In [ ]:
federated_algorithm = tff.templates.IterativeProcess(
      initialize_fn=initialize_fn,
      next_fn=next_fn
    )

In [ ]:
def evaluate(server_state):
  keras_model = generateAgePredictionResNet((121,145,121),paddingType = 'same',initType='he_uniform',regAmount=0.00005,dropRate=0.2)
  keras_model.compile(
      loss=tf.keras.losses.MeanAbsoluteError(),
      metrics=[tf.keras.metrics.MeanAbsoluteError()]
  )
  results = keras_model.evaluate(x_test, y_test, batch_size=5)

In [ ]:
server_state = federated_algorithm.initialize()

In [ ]:
state = iterative_process.initialize()

Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`


Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`


In [ ]:
evaluate(server_state)


10/10 [==============================] - 4s 166ms/step - loss: 58.2721 - mean_absolute_error: 58.2095


In [ ]:
for round in range(10):
  print("round ",round+1)
  server_state = federated_algorithm.next(server_state, federated_train_data)
  evaluate(server_state)
  print("")

round  1
10/10 [==============================] - 3s 165ms/step - loss: 46.2592 - mean_absolute_error: 46.1967

round  2


TypeError: ignored